In [ ]:
from pathlib import Path

import pandas as pd

path = Path("./data")

# Nodes
nodes = pd.read_csv(path / Path("Nodes/Nodes.csv"))
edges = pd.read_csv(path / Path("Edges/Edges (Product Sub-Group).csv"))

delivery_to_distributor = pd.read_csv(
    path / Path("Temporal Data/Unit/Delivery To distributor.csv")
)
factory_issue = pd.read_csv(path / Path("Temporal Data/Unit/factory issue.csv"))
production = pd.read_csv(path / Path("Temporal Data/Unit/Production .csv"))
sales_order = pd.read_csv(path / Path("Temporal Data/Unit/Sales order.csv"))


In [ ]:
import numpy as np
import pandas as pd
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

signals_path = path / Path("Temporal Data/Unit/")

production = pd.read_csv(signals_path / Path("Production .csv"))
factory_issue = pd.read_csv(signals_path / Path("factory issue.csv"))
delivery = pd.read_csv(signals_path / Path("Delivery To distributor.csv"))
sales_order = pd.read_csv(signals_path / Path("Sales order.csv"))

products = [col for col in production.columns if col != "Date"]
product_to_id = {prod: i for i, prod in enumerate(products)}


def transform_temporal(df, mapping):
    df_no_date = df.drop(columns=["Date"])
    return df_no_date.rename(columns=mapping)


X_prod = transform_temporal(production, product_to_id)
X_issue = transform_temporal(factory_issue, product_to_id)
X_delivery = transform_temporal(delivery, product_to_id)
X_sales = transform_temporal(sales_order, product_to_id)

X_prod_np = X_prod.to_numpy()
X_issue_np = X_issue.to_numpy()
X_delivery_np = X_delivery.to_numpy()
X_sales_np = X_sales.to_numpy()

# shape: [T, N, F] = [time_steps, num_nodes, num_features]
X = np.stack([X_prod_np, X_issue_np, X_delivery_np, X_sales_np], axis=-1)

y = X_sales_np[1:]
X = X[:-1]

print("X shape:", X.shape)  # (T-1, N, 4)
print("y shape:", y.shape)  # (T-1, N)

X shape: (220, 41, 4)
y shape: (220, 41)


In [ ]:
import torch
from torch.utils.data import Dataset


class GraphTimeSeriesDataset(Dataset):
    def __init__(self, X, y, window_size):
        """
        Args:
            X: np.ndarray or torch.Tensor of shape [T, N, F]
            y: np.ndarray or torch.Tensor of shape [T, N]
            window_size: int, number of timesteps per input window
        """
        if not torch.is_tensor(X):
            X = torch.tensor(X, dtype=torch.float32)
        if not torch.is_tensor(y):
            y = torch.tensor(y, dtype=torch.float32)

        self.X = X
        self.y = y
        self.window_size = window_size
        self.T, self.N, self.F = X.shape

        # total number of possible windows per node
        self.num_windows_per_node = self.T - self.window_size

    def __len__(self):
        # Each node has (T - window_size) samples
        return self.num_windows_per_node * self.N

    def __getitem__(self, idx):
        """
        Given a flat index, map to (node_id, time_index)
        """
        node_id = idx // self.num_windows_per_node
        t_start = idx % self.num_windows_per_node
        t_end = t_start + self.window_size

        x = self.X[t_start:t_end, node_id, :]  # [window_size, F]
        y = self.y[t_end - 1, node_id]  # scalar target (last timestep)
        return {"x": x, "y": y, "node_id": node_id}

In [ ]:
import torch.nn as nn


class LSTMBaseline(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # predict scalar target

    def forward(self, x):
        # x: [B, window_size, F]
        out, _ = self.lstm(x)  # out: [B, window_size, hidden_size]
        out = out[:, -1, :]  # take output of last timestep: [B, hidden_size]
        out = self.fc(out)  # [B, 1]
        return out.squeeze(-1)  # [B]

In [ ]:
from torch.utils.data import DataLoader


def evaluate_model(model, dataset, batch_size=64, criterion=None, device=None):
    model.eval()
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if criterion is None:
        criterion = nn.MSELoss()

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    total_loss = 0.0
    all_preds, all_trues = [], []

    with torch.no_grad():
        for batch in loader:
            x = batch["x"].to(device)
            y_true = batch["y"].to(device)
            y_pred = model(x)

            loss = criterion(y_pred, y_true)
            total_loss += loss.item() * x.size(0)

            all_preds.append(y_pred.cpu())
            all_trues.append(y_true.cpu())

    avg_loss = total_loss / len(dataset)
    all_preds = torch.cat(all_preds, dim=0)
    all_trues = torch.cat(all_trues, dim=0)
    rmse = torch.sqrt(torch.mean((all_preds - all_trues) ** 2))
    return avg_loss, rmse.item(), all_preds, all_trues


def train_lstm(
    model,
    train_dataset,
    val_dataset=None,
    window_size=12,
    batch_size=64,
    num_epochs=100,
    lr=0.001,
    eval_every=5,
    patience=10,
    save_path="best_model.pt",
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    best_val_loss = np.inf
    patience_counter = 0

    # Stats for plotting
    history = {
        "train_loss": [],
        "val_loss": [],
        "val_rmse": [],
        "best_val_loss": np.inf,
        "best_val_rmse": np.inf,
    }

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            x = batch["x"].to(device)
            y = batch["y"].to(device)

            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)

        avg_train_loss = total_loss / len(train_dataset)
        history["train_loss"].append(avg_train_loss)

        # Periodic evaluation
        if val_dataset is not None and epoch % eval_every == 0:
            val_loss, val_rmse, _, _ = evaluate_model(
                model, val_dataset, batch_size, criterion, device
            )
            history["val_loss"].append(val_loss)
            history["val_rmse"].append(val_rmse)
            print(
                f"Epoch {epoch}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}"
            )

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                torch.save(model.state_dict(), save_path)
                history["best_val_loss"] = best_val_loss
                history["best_val_rmse"] = val_rmse
                print(f" --> Best model saved at epoch {epoch}")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    break
        else:
            print(f"Epoch {epoch}/{num_epochs} | Train Loss: {avg_train_loss:.4f}")

    # Load best model
    if val_dataset is not None:
        model.load_state_dict(torch.load(save_path))

    return model, history

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

tscv = TimeSeriesSplit(n_splits=5)
splits = []
"""
{
    "k-fold": {
        model,
        best_val_loss,
        best_val_rmse
    }
}
"""
cv_results = {}

for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]

    # Prepare scaled containers
    X_train_scaled = np.zeros_like(X_train)
    y_train_scaled = np.zeros_like(y_train)
    X_test_scaled = np.zeros_like(X_test)
    y_test_scaled = np.zeros_like(y_test)

    scaler_X_per_product = []
    scaler_y_per_product = []

    T, N, F = X_train.shape
    for node_idx in range(N):
        # Fit a scaler for features of this product
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()

        X_node = X_train[:, node_idx, :]  # (T, F)
        y_node = y_train[:, node_idx].reshape(-1, 1)

        X_train_scaled[:, node_idx, :] = scaler_X.fit_transform(X_node)
        y_train_scaled[:, node_idx] = scaler_y.fit_transform(y_node).ravel()

        # Scale test data using same scaler
        X_test_scaled[:, node_idx, :] = scaler_X.transform(X_test[:, node_idx, :])
        y_test_scaled[:, node_idx] = scaler_y.transform(
            y_test[:, node_idx].reshape(-1, 1)
        ).ravel()

        scaler_X_per_product.append(scaler_X)
        scaler_y_per_product.append(scaler_y)

    window_size = int(
        0.1 * T
    )  # TODO: Consider here the implications of window size and the possibility of training a rolling LSTM with no fixed window size
    batch_size = 64

    train_dataset = GraphTimeSeriesDataset(
        X_train_scaled, y_train_scaled, window_size=window_size
    )
    test_dataset = GraphTimeSeriesDataset(
        X_test_scaled, y_test_scaled, window_size=window_size
    )

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    input_size = F  # number of features per node
    hidden_size = 16

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = LSTMBaseline(input_size, hidden_size).to(device)

    model, history = train_lstm(
        model,
        train_dataset,
        val_dataset=test_dataset,
        window_size=5,
        batch_size=batch_size,
        num_epochs=100,
        lr=0.001,
        eval_every=1,
        patience=20,
    )
    cv_results[f"fold_{i}"] = {
        "model": model,
        "best_val_loss": history["best_val_loss"],
        "best_val_rmse": history["best_val_rmse"],
    }

# # Print the size of each fold
# for i, (X_train_fold, X_test_fold, y_train_fold, y_test_fold) in enumerate(splits):
#     print(f"Fold {i+1}:")
#     print(f"Training set shape: {X_train_fold.shape}")
#     print(f"Test set shape: {X_test_fold.shape}")

Epoch 1/100 | Train Loss: 0.9565 | Val Loss: 3.2895 | Val RMSE: 1.8137
 --> Best model saved at epoch 1
Epoch 2/100 | Train Loss: 0.9324 | Val Loss: 3.2841 | Val RMSE: 1.8122
 --> Best model saved at epoch 2
Epoch 3/100 | Train Loss: 0.9213 | Val Loss: 3.2843 | Val RMSE: 1.8123
Epoch 4/100 | Train Loss: 0.9160 | Val Loss: 3.2838 | Val RMSE: 1.8121
 --> Best model saved at epoch 4
Epoch 5/100 | Train Loss: 0.9117 | Val Loss: 3.2859 | Val RMSE: 1.8127
Epoch 6/100 | Train Loss: 0.9068 | Val Loss: 3.2817 | Val RMSE: 1.8115
 --> Best model saved at epoch 6
Epoch 7/100 | Train Loss: 0.9006 | Val Loss: 3.2766 | Val RMSE: 1.8101
 --> Best model saved at epoch 7
Epoch 8/100 | Train Loss: 0.8950 | Val Loss: 3.2765 | Val RMSE: 1.8101
 --> Best model saved at epoch 8
Epoch 9/100 | Train Loss: 0.8877 | Val Loss: 3.2686 | Val RMSE: 1.8079
 --> Best model saved at epoch 9
Epoch 10/100 | Train Loss: 0.8793 | Val Loss: 3.2614 | Val RMSE: 1.8059
 --> Best model saved at epoch 10
Epoch 11/100 | Train Los

In [ ]:
import pandas as pd

# Convert cv_results to a DataFrame
results_df = pd.DataFrame(cv_results)

# Calculate mean and std of best_val_loss
mean_loss = results_df.loc["best_val_rmse"].mean()
std_loss = results_df.loc["best_val_rmse"].std()

print(f"Mean best validation loss: {mean_loss:.4f} ± {std_loss:.4f}")

Mean best validation loss: 1.2868 ± 0.7231


In [ ]:
# train = int(0.8 * X.shape[0])

# X_train, X_test = X[:train], X[train:]
# y_train, y_test = y[:train], y[train:]


# half_X, half_y = X[:int(0.5 * X.shape[0])], y[:int(0.5 * X.shape[0])]
# train = int(0.8 * half_X.shape[0])
# X_train, X_test = half_X[:train], half_X[train:]
# y_train, y_test = half_y[:train], half_y[train:]


In [19]:
from sklearn.preprocessing import StandardScaler

# Flatten for scaling: combine time and nodes
T, N, F = X_train.shape
X_flat = X_train.reshape(-1, F)  # shape [T*N, F]
y_flat = y_train.reshape(-1, 1)  # shape [T*N, 1]

# Create scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit and transform
X_train_scaled_flat = scaler_X.fit_transform(X_flat)
y_train_scaled_flat = scaler_y.fit_transform(y_flat)

# Reshape back to original shape
X_train_scaled = X_train_scaled_flat.reshape(T, N, F)
y_train_scaled = y_train_scaled_flat.reshape(T, N)

print(f"X_scaled range: {X_train_scaled.min():.2f} to {X_train_scaled.max():.2f}")
print(f"y_scaled range: {y_train_scaled.min():.2f} to {y_train_scaled.max():.2f}")


X_scaled range: -0.43 to 9.22
y_scaled range: -0.35 to 9.32


In [20]:
y_test_flat = y_test.reshape(-1, 1)
y_test_scaled_flat = scaler_y.transform(y_test_flat)
y_test_scaled = y_test_scaled_flat.reshape(y_test.shape)

X_test_flat = X_test.reshape(-1, F)
X_test_scaled_flat = scaler_X.transform(X_test_flat)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

T, N, F = X_train.shape

# Prepare scaled containers
X_train_scaled = np.zeros_like(X_train)
y_train_scaled = np.zeros_like(y_train)
X_test_scaled = np.zeros_like(X_test)
y_test_scaled = np.zeros_like(y_test)

scaler_X_per_product = []
scaler_y_per_product = []

for node_idx in range(N):
    # Fit a scaler for features of this product
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_node = X_train[:, node_idx, :]  # (T, F)
    y_node = y_train[:, node_idx].reshape(-1, 1)

    X_train_scaled[:, node_idx, :] = scaler_X.fit_transform(X_node)
    y_train_scaled[:, node_idx] = scaler_y.fit_transform(y_node).ravel()

    # Scale test data using same scaler
    X_test_scaled[:, node_idx, :] = scaler_X.transform(X_test[:, node_idx, :])
    y_test_scaled[:, node_idx] = scaler_y.transform(
        y_test[:, node_idx].reshape(-1, 1)
    ).ravel()

    scaler_X_per_product.append(scaler_X)
    scaler_y_per_product.append(scaler_y)


# LSTM baseline model

In [ ]:
from torch.utils.data import DataLoader

window_size = int(
    0.1 * T
)  # TODO: Consider here the implications of window size and the possibility of training a rolling LSTM with no fixed window size
batch_size = 64

train_dataset = GraphTimeSeriesDataset(
    X_train_scaled, y_train_scaled, window_size=window_size
)
test_dataset = GraphTimeSeriesDataset(
    X_test_scaled, y_test_scaled, window_size=window_size
)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [25]:
input_size = F  # number of features per node
hidden_size = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMBaseline(input_size, hidden_size).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model, history = train_lstm(
    model,
    train_dataset,
    val_dataset=test_dataset,
    window_size=5,
    batch_size=batch_size,
    num_epochs=100,
    lr=0.001,
    eval_every=1,
    patience=20,
)

Epoch 1/100 | Train Loss: 1.0161 | Val Loss: 0.6438 | Val RMSE: 0.8024
 --> Best model saved at epoch 1
Epoch 2/100 | Train Loss: 1.0064 | Val Loss: 0.6393 | Val RMSE: 0.7996
 --> Best model saved at epoch 2
Epoch 3/100 | Train Loss: 1.0039 | Val Loss: 0.6329 | Val RMSE: 0.7956
 --> Best model saved at epoch 3
Epoch 4/100 | Train Loss: 1.0008 | Val Loss: 0.6303 | Val RMSE: 0.7939
 --> Best model saved at epoch 4
Epoch 5/100 | Train Loss: 0.9967 | Val Loss: 0.6146 | Val RMSE: 0.7840
 --> Best model saved at epoch 5
Epoch 6/100 | Train Loss: 0.9912 | Val Loss: 0.5998 | Val RMSE: 0.7744
 --> Best model saved at epoch 6
Epoch 7/100 | Train Loss: 0.9797 | Val Loss: 0.5958 | Val RMSE: 0.7719
 --> Best model saved at epoch 7
Epoch 8/100 | Train Loss: 0.9566 | Val Loss: 0.5807 | Val RMSE: 0.7620
 --> Best model saved at epoch 8
Epoch 9/100 | Train Loss: 0.9328 | Val Loss: 0.5394 | Val RMSE: 0.7344
 --> Best model saved at epoch 9
Epoch 10/100 | Train Loss: 0.9176 | Val Loss: 0.5294 | Val RMSE:

In [ ]:
all_preds = []
all_trues = []
all_node_ids = []

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

with torch.no_grad():
    for batch in test_loader:
        x = batch["x"].to(device)  # [B, window_size, F]
        y_true = batch["y"].to(device)  # [B]
        node_ids = batch["node_id"]  # [B], keep on CPU

        y_pred = model(x)

        all_preds.append(y_pred.cpu())
        all_trues.append(y_true.cpu())
        all_node_ids.append(node_ids)

In [ ]:
num_examples = 10
print("Example predictions on test set:")
for i in range(num_examples):
    print(
        f"Node {all_node_ids[i].item()} | True: {all_trues[i].item():.2f} | Pred: {all_preds[i].item():.2f}"
    )

Example predictions on test set:
Node 4 | True: -1.21 | Pred: -0.67
Node 1 | True: 0.29 | Pred: 0.25
Node 40 | True: -0.08 | Pred: 0.01
Node 2 | True: 0.45 | Pred: -0.67
Node 31 | True: 2.18 | Pred: 0.50
Node 28 | True: -0.43 | Pred: -0.54
Node 7 | True: -0.68 | Pred: 0.14
Node 18 | True: -0.84 | Pred: -0.17
Node 27 | True: 1.14 | Pred: 0.45
Node 27 | True: -0.77 | Pred: -0.60


# GCNLSTM baseline model

In [ ]:
edges = pd.read_csv(
    "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Edges/Edges (Plant).csv"
)

edges["node1"] = edges["node1"].astype(str)
edges["node2"] = edges["node2"].astype(str)

edges["node1"] = edges["node1"].map(product_to_id)
edges["node2"] = edges["node2"].map(product_to_id)

edge_index = edges[["node1", "node2"]].to_numpy().T.astype(np.int64)

num_nodes = len(product_to_id)

train_dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=np.ones(edge_index.shape[1]),
    features=X_train_scaled,
    targets=y_train_scaled,
)

test_dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=np.ones(edge_index.shape[1]),
    features=X_test_scaled,
    targets=y_test_scaled,
)


In [129]:
# from torch_geometric_temporal.signal import temporal_signal_split

# train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GCLSTM


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GCLSTM(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)  # Unpack (H, C)
        h = F.relu(h)
        h = self.linear(h)
        return h.squeeze(-1)

In [ ]:
model = RecurrentGCN(node_features=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from tqdm import tqdm


def calculate_mse(model, dataset):
    """Calculate MSE on a dataset"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for time, snapshot in enumerate(dataset):
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            total_loss += torch.mean((y_hat - snapshot.y) ** 2).item()
    model.train()
    return total_loss / (time + 1)


n_epoch = 100
for epoch in tqdm(range(n_epoch)):
    model.train()
    total_loss = 0
    for time, snapshot in enumerate(train_dataset):
        optimizer.zero_grad()
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        loss = torch.mean((y_hat - snapshot.y) ** 2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        train_mse = calculate_mse(model, train_dataset)
        test_mse = calculate_mse(model, test_dataset)
        print(f"\nEpoch {epoch + 1}/{n_epoch}")
        print(f"Train MSE: {train_mse:.4f}")
        print(f"Test MSE: {test_mse:.4f}")


 10%|█         | 10/100 [00:10<01:43,  1.15s/it]


Epoch 10/100
Train MSE: 0.9739
Test MSE: 2.4134


 20%|██        | 20/100 [00:21<01:33,  1.17s/it]


Epoch 20/100
Train MSE: 0.9727
Test MSE: 2.4125


 30%|███       | 30/100 [00:32<01:20,  1.15s/it]

...


Epoch 80/100
Train MSE: 0.9659
Test MSE: 2.4127


 90%|█████████ | 90/100 [01:36<00:12,  1.21s/it]


Epoch 90/100
Train MSE: 0.9652
Test MSE: 2.4130


100%|██████████| 100/100 [01:48<00:00,  1.09s/it]


Epoch 100/100
Train MSE: 0.9650
Test MSE: 2.4128
